In [1]:
import torch
import torch.optim as optim
import torch.nn as nn
from tqdm import tqdm
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from torch_geometric.loader import DataLoader
import matplotlib.pyplot as plt
import optuna  # 【新增】导入 Optuna

# ==============================================================================
#                      【自定义模块导入】 (保持不变)
# ==============================================================================
# 请确保这些文件在您的路径下
from loss_function.GNN_LSTM_with_Attention_V2 import GNN_LSTKAN_with_Attention_v2, GNNEncoder
from Slide_Window_and_Graph import create_graph_list_from_df
from kan_improved import KAN
from loss_function.custom_lossess_v4 import jerk_smoothness_loss_v2, velocity_heading_consistency_loss_v2

# ==============================================================================
#                      全局设置与数据准备
# ==============================================================================
# 将数据加载放在全局，避免每次 Optuna 尝试时重复加载
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# filesname = "Cheku.csv" # 或者根据您的实际变量名

# --- 1. 数据加载 ---
# 假设数据已经存在，这里保留您的读取逻辑
# 请根据实际情况取消注释或修改路径
# try:
#     train_data = pd.read_csv(r'Datasets/x_train.csv')
#     val_data = pd.read_csv(r'Datasets/x_val.csv')
#     test_data = pd.read_csv(r'Datasets/x_test.csv')
#     print("数据加载成功！")
# except FileNotFoundError:
#     print("【警告】未找到数据集文件，请检查路径。代码将无法运行数据部分。")
#     # 为了演示代码结构，这里模拟数据 (仅用于调试，实际运行时请删除)
#     # train_data = pd.DataFrame(np.random.rand(100, 10), columns=[f'col_{i}' for i in range(10)])
#     # val_data = pd.DataFrame(np.random.rand(20, 10), columns=[f'col_{i}' for i in range(10)])
#     # test_data = pd.DataFrame(np.random.rand(20, 10), columns=[f'col_{i}' for i in range(10)])
#     pass


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
direct = "Datasets/"
filesname = "0.15_Speed_withoutOB.csv"
print(f"filename is {filesname}")
data = pd.read_csv(direct + filesname)
# 2. 定义分割比例
train_ratio = 0.8
val_ratio = 0.1
# # 剩下的 10% 将作为测试集

# # 3. 计算分割点索引 (按时间顺序)
data_size = len(data)
train_end_idx = int(train_ratio * data_size)
val_end_idx = train_end_idx + int(val_ratio * data_size)

# # # 4. 使用 .iloc 分割 DataFrame
train_data = data.iloc[:train_end_idx]
val_data = data.iloc[train_end_idx:val_end_idx]
test_data = data.iloc[val_end_idx:]

# --- 2. 特征工程 ---
column_names = train_data.drop(['timestamp','x_coord', 'y_coord'], axis=1).columns.tolist()
coord_cols = ['x_coord', 'y_coord']
wifi_features = [col for col in column_names if any(sensor in col for sensor in ["RSSI","distance","rot"])]
imu_features = [col for col in column_names if any(sensor in col for sensor in ["accelerometer", "gyroscope","magnetometer"])]

scaler_wifi = StandardScaler().fit(train_data[wifi_features])
scaler_imu = StandardScaler().fit(train_data[imu_features])
scaler_coords = StandardScaler().fit(train_data[coord_cols])

train_df_scaled = train_data.copy()
val_df_scaled = val_data.copy()
test_df_scaled = test_data.copy()

for df in [train_df_scaled, val_df_scaled, test_df_scaled]:
    df[wifi_features] = scaler_wifi.transform(df[wifi_features])
    df[imu_features] = scaler_imu.transform(df[imu_features])
    df[coord_cols] = scaler_coords.transform(df[coord_cols])

# --- 3. 图构建 ---
windows_size = 60
future_steps = 2
future_radius = 10
past_radius = 20
batch_size = 128  # 这个也可以作为超参数搜索，但通常固定为 64 或 128

print("正在构建图数据 (只需一次)...")
train_data_list = create_graph_list_from_df(train_df_scaled, wifi_features, imu_features, windows_size, future_steps, 'cpu', future_radius, past_radius)
val_data_list = create_graph_list_from_df(val_df_scaled, wifi_features, imu_features, windows_size, future_steps, 'cpu', future_radius, past_radius)
# test_data_list 在 Optuna 搜索时不需要，最后评估才用

train_loader = DataLoader(train_data_list, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_data_list, batch_size=batch_size, shuffle=False)

#al 剪枝)
# # ==============================================================================
# 
# def train_phase_1_optuna(model, train_loader, val_loader, lr, weight_decay, epochs, patience, kan_reg_weight, lambda_vel, trial):
#     """
#     第一阶段训练 - 适配 Optuna
#     """
#     criterion = nn.HuberLoss()
#     optimizer = optim.AdamW(model.parameters(),)"r=lr, weight_decay=weight_decay)
#     scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor=0.5, patience=10)
#     
#     best_val_l 全局维度定义
wifi_feat_dim = len(wifi_features)
imu_feat_dim = len(imu_features)
kan_output_dim = future_steps * 2


# ==============================================================================
#                      修正后的训练函数 (解决 AttributeError)
# ==============================================================================

def train_phase_1_optuna(model, train_loader, val_loader, lr, weight_decay, epochs, patience, kan_reg_weight, lambda_vel, trial):
    """
    第一阶段训练 - 修正版
    返回: best_model_state, best_val_loss, actual_epochs_run
    """
    criterion = nn.HuberLoss()
    optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor=0.5, patience=10)
    
    best_val_loss = float('inf')
    best_model_state = None
    patience_counter = 0
    actual_epochs_run = 0 # 记录实际运行了多少轮
    
    for epoch in range(epochs):
        actual_epochs_run = epoch + 1 # 更新当前轮数
        model.train()
        for batch in train_loader:
            batch = batch.to(device)
            optimizer.zero_grad()
            main_pred, _, _ = model(batch)
            true_seq = batch.y.view(batch.num_graphs, future_steps, 2)
            
            loss_coord = criterion(main_pred, true_seq)
            pred_vel = (main_pred[:, 1:] - main_pred[:, :-1])
            true_vel = (true_seq[:, 1:] - true_seq[:, :-1])
            loss_vel = criterion(pred_vel, true_vel)
            
            reg_loss = model.regularization_loss()
            total_loss = loss_coord + lambda_vel * loss_vel + kan_reg_weight * reg_loss
            
            total_loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
        
        # Validation
        model.eval()
        val_loss_accum = 0
        with torch.no_grad():
            for batch in val_loader:
                batch = batch.to(device)
                pred_seq, _, _ = model(batch)
                true_seq = batch.y.view(batch.num_graphs, future_steps, 2)
                
                loss_coord = criterion(pred_seq, true_seq)
                pred_vel = (pred_seq[:, 1:] - pred_seq[:, :-1])
                true_vel = (true_seq[:, 1:] - true_seq[:, :-1])
                loss_vel = criterion(pred_vel, true_vel)
                
                val_loss_accum += (loss_coord + lambda_vel * loss_vel).item()
        
        avg_val_loss = val_loss_accum / len(val_loader)
        scheduler.step(avg_val_loss)

        # --- Optuna Pruning ---
        trial.report(avg_val_loss, epoch)
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            best_model_state = model.state_dict() # 保存权重副本
            patience_counter = 0
        else:
            patience_counter += 1
            
        if patience_counter >= patience:
            break
            
    # 返回实际运行的轮数，供第二阶段接续
    return best_model_state, best_val_loss, actual_epochs_run

def train_phase_2_optuna(model, train_loader, val_loader, lr, weight_decay, epochs, patience, smooth_weight, consistency_weight, lambda_vel, trial, start_step):
    """
    第二阶段训练 - 修正版
    新增参数: start_step (来自第一阶段的轮数)
    """
    criterion = nn.HuberLoss()
    optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor=0.5, patience=5)
    
    best_val_loss = float('inf')
    patience_counter = 0

    for epoch in range(epochs):
        model.train()
        for batch in train_loader:
            batch = batch.to(device)
            optimizer.zero_grad()
            main_pred, _, _ = model(batch)
            true_seq = batch.y.view(batch.num_graphs, future_steps, 2)
            
            loss_coord = criterion(main_pred, true_seq)
            pred_vel = (main_pred[:, 1:] - main_pred[:, :-1])
            true_vel = (true_seq[:, 1:] - true_seq[:, :-1])
            loss_vel = criterion(pred_vel, true_vel)
            
            base_loss = loss_coord + lambda_vel * loss_vel
            smooth_loss = jerk_smoothness_loss_v2(main_pred)
            consistency_loss = velocity_heading_consistency_loss_v2(main_pred)
            
            total_loss = base_loss + smooth_weight * smooth_loss + consistency_weight * consistency_loss
            
            total_loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            
        # Validation
        model.eval()
        val_loss_accum = 0
        with torch.no_grad():
            for batch in val_loader:
                batch = batch.to(device)
                pred_seq, _, _ = model(batch)
                true_seq = batch.y.view(batch.num_graphs, future_steps, 2)
                
                loss_coord = criterion(pred_seq, true_seq)
                pred_vel = (pred_seq[:, 1:] - pred_seq[:, :-1])
                true_vel = (true_seq[:, 1:] - true_seq[:, :-1])
                loss_vel = criterion(pred_vel, true_vel)
                
                val_loss_accum += (loss_coord + lambda_vel * loss_vel).item()
        
        avg_val_loss = val_loss_accum / len(val_loader)
        scheduler.step(avg_val_loss)

        # Optuna Reporting (接续第一阶段的 step)
        current_total_step = start_step + epoch
        trial.report(avg_val_loss, current_total_step)
        
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            patience_counter = 0
        else:
            patience_counter += 1
        
        if patience_counter >= patience:
            break
            
    return best_val_loss

# ==============================================================================
#                      Optuna Objective Function (修正版)
# ==============================================================================

def objective(trial):
    # --- 1. 采样超参数 ---
    # 模型架构参数
    gnn_hidden = trial.suggest_categorical("gnn_hidden_dim", [32, 64, 96, 128])
    gnn_heads = trial.suggest_categorical("gnn_heads", [2, 4, 8])
    dropout = trial.suggest_float("dropout",0, 0.5)
    
    # 【新增】LSTM 相关的搜索参数
    lstm_hidden = trial.suggest_categorical("lstm_hidden", [32, 64, 96,28])
    lstm_num_layers = trial.suggest_int("lstm_num_layers", 1, 3) # 搜索 1, 2, 3 层
    gnn_num_layers = trial.suggest_int("gnn_num_layers", 1, 3)
    kan_middle_dim = trial.suggest_int("kan_middle_dim", 32, 256, step=16)
    
    # Phase 1 参数
    lr_p1 = trial.suggest_float("lr_phase1", 1e-4, 5e-3, log=True)
    weight_decay = trial.suggest_float("weight_decay", 1e-5, 1e-3, log=True)
    kan_reg_w = trial.suggest_float("kan_reg_weight", 1e-5, 1e-3, log=True)
    lambda_vel = trial.suggest_float("lambda_vel",0.001, 2, log=True)
    
    # Phase 2 参数
    lr_p2 = trial.suggest_float("lr_phase2", 1e-5, 1e-3, log=True) 
    smooth_w = trial.suggest_float("smooth_weight", 1e-5, 1e-3, log=True)
    consistency_w = trial.suggest_float("consistency_weight", 1e-5, 1e-3, log=True)

    # --- 2. 实例化模型 ---
    gnn_encoder = GNNEncoder(
        wifi_input_dim=wifi_feat_dim,
        imu_input_dim=imu_feat_dim,
        hidden_dim=gnn_hidden,
        windows_size=windows_size,
        num_layers=gnn_num_layers,
        heads=gnn_heads,
        dropout=dropout
    ).to(device)

    kan_layers_config = [gnn_hidden, kan_middle_dim, 16, kan_output_dim]
    
    # 【注意】这里假设您的 KAN 类支持 lstm_num_layers 参数
    # 如果您的 KAN 类中 LSTM 层数参数名是 num_layers，请相应修改
    kan_predictor = KAN(
        layers_hidden=kan_layers_config,
        use_lstm=True,
        lstm_hidden=lstm_hidden,
        
        # 【新增】传入搜索到的 LSTM 层数
        # 请检查您的 kan_improved.py 文件，确认控制 LSTM 层数的参数名是否为 lstm_num_layers
        # 如果您的代码中是用 num_layers 控制 LSTM，则写成 num_layers=lstm_num_layers
        dropout_rate=dropout,
        num_layers=lstm_num_layers, # 这里保留原有的 num_layers (假设它是控制 KAN 结构的层数)
        grid_size=5,
        spline_order=3,
    ).to(device)

    model = GNN_LSTKAN_with_Attention_v2(
        gnn_encoder=gnn_encoder,
        kan_predictor=kan_predictor,
        future_steps=future_steps
    ).to(device)

    # --- 3. 执行 Phase 1 训练 ---
    p1_epochs = 20
    p1_patience = 15
    
    best_state_p1, _, epochs_run_p1 = train_phase_1_optuna(
        model, train_loader, val_loader, 
        lr=lr_p1, weight_decay=weight_decay, 
        epochs=p1_epochs, patience=p1_patience, 
        kan_reg_weight=kan_reg_w, lambda_vel=lambda_vel,
        trial=trial
    )
    
    if best_state_p1 is None:
        raise optuna.exceptions.TrialPruned()

    # --- 4. 执行 Phase 2 训练 ---
    model.load_state_dict(best_state_p1)
    
    p2_epochs = 100
    p2_patience = 15
    
    final_val_loss = train_phase_2_optuna(
        model, train_loader, val_loader,
        lr=lr_p2, weight_decay=weight_decay,
        epochs=p2_epochs, patience=p2_patience,
        smooth_weight=smooth_w, consistency_weight=consistency_w,
        lambda_vel=lambda_vel, trial=trial,
        start_step=epochs_run_p1 
    )
    
    return final_val_loss
# ==============================================================================
#                      主程序入口
# ==============================================================================

if __name__ == "__main__":
    # 创建 Study
    # direction="minimize" 表示我们要最小化 loss
    # pruner=optuna.pruners.MedianPruner() 表示如果某次实验结果比之前实验的中位数还差，就停止
    study = optuna.create_study(direction="minimize", pruner=optuna.pruners.MedianPruner(n_warmup_steps=10))
    
    print("开始 Optuna 自动调参...")
    # n_trials=50 表示总共尝试 50 组参数组合
    study.optimize(objective, n_trials=30)

    print("\n" + "="*50)
    print("Optuna 搜索完成！")
    print("="*50)
    
    print("Best trial:")
    trial = study.best_trial
    print(f"  Value (Best Val Loss): {trial.value}")
    print("  Params: ")
    for key, value in trial.params.items():
        print(f"    {key}: {value}")

    # =================================================================
    #      (可选) 使用搜索到的最佳参数重新训练最终模型并保存
    # =================================================================
    # 如果您想直接跑出最终结果，可以在这里解析 trial.params，
    # 然后用这些参数重新实例化模型并调用您原始的 train_phase_1 和 train_phase_2 函数。
    
    # 简单的保存最佳参数到文件
    best_params_df = pd.DataFrame([trial.params])
    best_params_df.to_csv("optuna_best_params.csv", index=False)
    print("最佳参数已保存至 optuna_best_params.csv")
    optuna.visualization.plot_optimization_history(study).show()
    optuna.visualization.plot_param_importances(study).show() # 查看哪个参数最重要

c:\Users\Desktop\AppData\Local\Programs\Python\Python314\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


filename is 0.15_Speed_withoutOB.csv
正在构建图数据 (只需一次)...


[I 2025-12-07 16:57:20,785] A new study created in memory with name: no-name-cec9c37e-a458-4740-a673-d5f9566808df
c:\Users\Desktop\AppData\Local\Programs\Python\Python314\Lib\site-packages\torch\nn\modules\rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1760641939026789 and num_layers=1
  warnings.warn(


开始 Optuna 自动调参...


[I 2025-12-07 17:09:19,389] Trial 0 finished with value: 0.01145548671907322 and parameters: {'gnn_hidden_dim': 96, 'gnn_heads': 8, 'dropout': 0.1760641939026789, 'lstm_hidden': 28, 'lstm_num_layers': 1, 'gnn_num_layers': 3, 'kan_middle_dim': 176, 'lr_phase1': 0.00018973502509167045, 'weight_decay': 3.1897671798933224e-05, 'kan_reg_weight': 0.0003285476682340779, 'lambda_vel': 0.0012345841886109488, 'lr_phase2': 1.7931055964301815e-05, 'smooth_weight': 0.0001402689433861436, 'consistency_weight': 0.0007751104912733603}. Best is trial 0 with value: 0.01145548671907322.
[I 2025-12-07 17:12:40,921] Trial 1 finished with value: 0.006127012830058282 and parameters: {'gnn_hidden_dim': 64, 'gnn_heads': 8, 'dropout': 0.06229604281983908, 'lstm_hidden': 28, 'lstm_num_layers': 2, 'gnn_num_layers': 2, 'kan_middle_dim': 96, 'lr_phase1': 0.0014859238111918848, 'weight_decay': 7.548720320713535e-05, 'kan_reg_weight': 0.00024844089388430593, 'lambda_vel': 0.1583765656505329, 'lr_phase2': 0.0006400005


Optuna 搜索完成！
Best trial:
  Value (Best Val Loss): 0.004622921633364802
  Params: 
    gnn_hidden_dim: 32
    gnn_heads: 2
    dropout: 0.4011055079602454
    lstm_hidden: 28
    lstm_num_layers: 1
    gnn_num_layers: 1
    kan_middle_dim: 128
    lr_phase1: 0.002739081513013747
    weight_decay: 3.4554602832872516e-05
    kan_reg_weight: 4.17031854750352e-05
    lambda_vel: 0.029230732276031263
    lr_phase2: 3.7927820632194276e-05
    smooth_weight: 1.0252192225568273e-05
    consistency_weight: 0.00033433718376037804
最佳参数已保存至 optuna_best_params.csv


ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

In [3]:
import matplotlib.pyplot as plt
optuna.visualization.plot_optimization_history(study).show()
optuna.visualization.plot_param_importances(study).show() # 查看哪个参数最重要

ImportError: Tried to import 'plotly' but failed. Please make sure that the package is installed correctly to use this feature. Actual error: No module named 'plotly'.